A WebsiteData object to make code generaizable and easier to use with just URL's

In [1]:
from bs4 import BeautifulSoup
import urllib
import re
import nbimporter
import getTextAndFiles as helper

class WebsiteData:
    
    def __init__(self, url, nodeParams, keyValues):
        html_page = urllib.request.urlopen(url)
        self.soup = BeautifulSoup(html_page, "lxml")
        #self.URL =  self.getUsefulParentURL(url)
        self.URL = url
        self.links = self.getUsefulLinks()
        self.nodeParameters = nodeParams
        self.keyValueLists = keyValues
        self.JSONNode = {}
            
    #Need to adjust parent URL to ensure that it can be used to filter the links in the page
    def getUsefulParentURL(self, parentURL):
        #list of possible correct endings for the parentURL
        possibleParentEnds = ['.org', '.edu', '.com']
    
        for end in possibleParentEnds:
            #attempts to split parentURL by one of the endings
            parentURLSplit = parentURL.split(end)
        
            #if the split is successful then the parentURL is reassigned to the string before the split + the ending
            if len(parentURLSplit) > 1:
                parentURL = parentURLSplit[0] + end
                #print('accessed')
                return parentURL
            else:
                return 'invalid string'
            
    def urlJoin(self, parentURL, end):
        if end[0] == '/':
            return parentURL + end
        else:
            return parentURL + '/' + end
            
    #Finds all links in the page that are within the website and not malformed by ensuring the parentURL is in the URL.
    def getUsefulLinks(self):
        links = helper.getLinks(self.URL, 5)
    
        #if self.soup.find_all('a') != None:
         #   for link in self.soup.find_all('a'):
          #      if link != None and link.get('href') != None and self.URL in link.get('href'):
                    #print(link.get('href'))
           #         links.append(link.get('href'))
        
        #links.append(self.URL)
        #return links
        #pattern = re.compile("((http|ftp)s?://.*?)")
    
        #for child in self.soup.recursiveChildGenerator():
        #    name = getattr(child, "name", None)
        #    if name is not None:
        #        if name == 'a':
        #            if child.get('href') != None:
        #                current_link = child.get('href')
        #                if not pattern.match(current_link):
        #                    current_link = self.urlJoin(self.URL, current_link)
        #                if self.URL in current_link:
        #                    links.append(current_link)
        #links.append(self.URL)
        return links
    
    #Finds all tags within a BeautifulSoup object that's text contains a certain keyword and returns a list of these tags.
    def findTags(self, soup, keyWord):
        tagList = []
        
        for elem in soup(text=re.compile(keyWord)):
            tagList.append(elem.parent)
            
        return tagList

    #Utilizes a BeautifulSoup object (soup) to scrape all text that contains the given keyword. Filters for certain tags and does
    #not take text from solely header (h) tags, instead takes text from next filtered tag after the header. Returns 
    def keyWordScraper(self, soup, keyWord):
        searchTxt = ''
        correctName = ['p', 'li', 'table', 'ul', 'ol']
        tagList = self.findTags(soup, keyWord)
    
        #This loop goes through each tag and rips the text from that tag. If that tag is a header, then the text from the next 
        #useful tag is ripped instead. All text is saved in searchTxt
        for tag in tagList:
            if 'h' in tag.name:
                typeTag = type(soup.find('li'))
                current = tag.next_sibling
                while current != None and  not isinstance(current, typeTag) and current.name not in correctName:
                    current = current.next_sibling
                if current != None:
                    if 'ul' in current.name or 'ol' in current.name:
                        for li in current.findAll('li'):
                            searchTxt = searchTxt + li.text
                    else:
                        searchTxt = searchTxt + current.text
            #elif url extension includes the keyWord, grab all the text? only if NO keywords found on that page? 
            else:
                if(tag.name in correctName):
                    searchTxt = searchTxt + tag.text
                
        return searchTxt
    
    #Applies the keyWordScraper function to a BeautifulSoup object(soup) for a list of keywords. Returns the text found.
    def mapKeywordScraper(self, soup, keyWords):
        foundTxt = ''
    
        for keyWord in keyWords:
            foundTxt = foundTxt + self.keyWordScraper(soup, keyWord)
        #print(foundTxt)
        return foundTxt
    
    #this method takes a list of links and applies the scraping method to that list. 
    #WARNING: this method may take a while since it goes through every link, give it time to run before terminating it.
    def mapURLs(self, keywords):
        foundTxt = ''
        for link in self.links:
#            try:
#                new_page = urllib.request.urlopen(link)
#                new_soup = BeautifulSoup(new_page, "lxml")
#                foundTxt = foundTxt + self.mapKeywordScraper(new_soup, keywords)
#                print(foundTxt)
#            except urllib.error.HTTPError as err:
#                if err.code == 404:
#                    testedLinks[links[i]] = 'Page not found! (404 Error)' 
#                elif err.code == 403:
#                    testedLinks[links[i]] = 'Access denied! (403 Error)'
#                else:
#                    testedLinks[links[i]] = 'Something happened! Error code: ' + err.code
            new_page = urllib.request.urlopen(link)
            new_soup = BeautifulSoup(new_page, "lxml")
            foundTxt = foundTxt + self.mapKeywordScraper(new_soup, keywords)
        return foundTxt
    
    #creates JSON object
    def createJsonNode(self):
        for x in range(0, len(self.keyValueLists)):
            self.JSONNode[self.nodeParameters[x]] = [self.mapURLs(self.keyValueLists[x])]
            print(self.JSONNode)
        return self.JSONNode
        

Importing Jupyter notebook from getTextAndFiles.ipynb
